<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/machine_learning_pills/01_supervised/07_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Credit Card Fraud Detection

### Dataset Sbilanciati

<br>

![fraud](https://www.techexplorist.com/wp-content/uploads/2018/09/MIT-Fraud-Detection.jpg)

<br>

[Image Credits](https://www.techexplorist.com/false-positive-reduction-credit-card-fraud-detection/17191/)

<br>

### Obiettivo del notebook

L'obiettivo di questo notebook è proprio la scoperta di frodi su carte di credito in un dataset molto sbilanciato.

Quando si tratta di individuare frodi, guasti o comunque eventi rari i nostri dataset saranno probabilmente molto sbilanciati, ovvero una classe sarà molto più rappresentata delle altre.

In questo caso il nostro stimatore potrebbe non individuare nessun pattern e classificare tutte le osservazioni nella medesima classe. Per ovviare a questo problema si possono usare le segiuenti tecniche:
* **undersampling**: eliminare molte osservazioni della classe più rappresentata in modo da bilanciare il dataset. Consigliato quando si hanno molte osservazioni.

* **oversampling**: duplicare le osservazioni della classe con eventi rari per bilanciare il dataset. Io personalmente non consiglio questa strategia, si introduce molto rumore che potrebbe portare ad overfitting, se proprio non si può applicare l'undersampling il mio consiglio è di fare oversampling *mitigando* lo sbilancimento, ad esempio se il rapporto è 99% - 1% provare prima a portarlo a 90% - 10%.

<br>

![oversampling](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn-images-1.medium.com%2Fmax%2F1600%2F1*P93SeDGPGw0MhwvCcvVcXA.png&f=1&nofb=1)

<br>

[Image Credits](https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb)

<br>

Per applicare queste trasformazioni in python esiste una libreria, [imblearn](https://imbalanced-learn.readthedocs.io/en/stable/api.html) che mette a disposizione molti tipi di bilanciamento. Nel nostro caso applicheremo un bilanciamento casuale delle osservazioni quindi non la useremo.

Come stimatore verrà usata la regressione logistica.

<br>

![penguin](https://miro.medium.com/max/800/1*UgYbimgPXf6XXxMy2yqRLw.png)

<br>

[Image Credits](http://dataaspirant.com/2017/03/02/how-logistic-regression-model-works/)

<br>

### Il Dataset

Il dataset utilizzato è stato generato tramite Paysim, un simulatore di transazioni di carte di credito.

Per motivi di spazio di github ne carico solo una minima parte, [l'intero dataset può essere scaricato da kaggle](https://www.kaggle.com/ntnu-testimon/paysim1).

I campi del dataset sono:

* step: simulazione dell'unità di tempo
* type: il tipo di movimento. CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.
* amount: ammontare della transazione
* nameOrig: cliente che ha iniziato la transazione
* oldbalanceOrg: saldo iniziale al momento della transazione
* newbalanceOrig: saldo dopo la transazione
* nameDest: destinatario della transazione
* oldbalanceDest: saldo iniziale del ricevente
* newbalanceDest: saldo del ricevente dopo la transazione
* isFraud: variabile binaria che etichetta la frode (1) e il movimento regolare (0)
* isFlaggedFraud: variabile binaria che controlla le anomalie, per anomalia in questo caso si intende un movimento di oltre 200K in una singola transazione.

### Caricamento del dataset

***Se stai usando il notebook su Colab esegui le prossime due celle, altrimenenti vai direttamente al caricamento con *read_csv* inserendo il path del tuo file *Paysim.csv***

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/YOURPATH'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Paysim.csv')

In [0]:
df = pd.read_csv("Paysim.csv")
df.head()

### Analisi del dataset

Per prima cosa vediamo la percentuale di frodi nel dataset

In [0]:
fraud_count = df.isFraud.value_counts()

print("Not Fraud: ", fraud_count[0])
print("Fraud: ", fraud_count[1])
print("\n")
print("Percentuale frodi: ", round(fraud_count[1]/fraud_count.sum())*100,2, "%")

Il numero di frodi è bassissimo ma fortunatamente abbiamo abbastanza casi per applicare l'undersampling.

Iniziamo col vedere che alcune variabili non sono informative per noi, possono essere tranquillamente eliminate:
* step
* nameOrig
* nameDest

In [0]:
df.drop(columns = ['step', 'nameOrig', 'nameDest'], inplace = True)

### Analisi delle variabili

Esercizio. Cercate di trovare relazioni nascoste tra i dati.

* Influenza sulla variabile target.
* Relazioni tra i regressori.
* Analisi grafica

Vedremo a lezione se con le soluzioni proposte il modello avrà performance migliori.

### Preprocessing

#### Missing Values

Non sono presenti valori mancanti (il dataset è frutto di un generatore).

In [0]:
df.info()

#### Trasformazioni

Abbiamo tre tipologie di variabili:
* binaria
* numerica continua
* categorica

Per la variabile binaria non c'è nessun tipo di traformazione da fare.

Per le variabili numeriche applicheremo una normalizzazione per portarle tutte nel range [0,1].

Studiamo la variabile categorica per capire che trasformazione applicare.

In [0]:
df.type.value_counts()

*type* ha 4 valori, quindi possiamo applicare un One-Hot Encoding.

Per applicare le trasformazioni useremo uno strumento che rende il codice riutilizzabile ed elegante, il [ColumnTransormer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

In [0]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [0]:
# scelta degli attributi

cat_attribs = ["type"]
num_attribs = list(df._get_numeric_data().columns[0:-2]) # prendi tutte le colonne di tipo numerico e restituisce i nomi, tranne le ultime due

Il ColumnTransformer non è altro che una lista di operazioni da compiere, ogni elemento della lista è composto da:
* nome del passo
* traformazione da applicare
* colonne su cui applicarla

La totale integrazione con scikit-learn fa si che questo elemento abbia gli stessi metodi che abbiamo visto fino ad ora:
* fit
* transform
* fit_transform

e basta applicarlo a tutto il dataset, questo riduce di molto la probabilità di errore dovute a replicazione del codice.

In [0]:
transform = ColumnTransformer([
                               ("num", MinMaxScaler(), num_attribs),
                               ("cat", OneHotEncoder(), cat_attribs),
                               
])

In [0]:
df_prepared = transform.fit_transform(df)
df_prepared[0]

Il trasformer ci restituisce una matrice che dovremo riconvertire in DataFrame per applicare le successive operazioni.

In [0]:
df_complete = pd.concat([pd.DataFrame(df_prepared), df[["isFraud", "isFlaggedFraud"]]], axis =1 )

In [0]:
df_complete.head()

# esercizio: mettere il nome alle colonne

### Modelling: dataset sbilanciato

* baseline model
* reglog

In [0]:
from sklearn.model_selection import train_test_split

X = df_complete.drop("isFraud", axis=1).copy()
y = df_complete["isFraud"]

Visto che il dataset è fortemente sbilanciato applichiamo un campionamento stratificato sulla variabile target così da evitare che il train (o il test) set non presentino nessuna frode.

In [0]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 42)

#### Baseline model

Il baseline model è il nostro punto di partenza, in questo caso noi vogliamo un modello che classifica sempre la stessa etichetta. In questo caso l'accuratezza sarà altissima ma a noi interessa confrontarci su altre metriche:
* precision
* recall

In [0]:
from sklearn.dummy import DummyClassifier

dc = DummyClassifier(strategy = "most_frequent")

dc.fit(train_x, train_y)

yhat_dc = dc.predict(test_x)

In [0]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(train_x, train_y)

yhat_lr = lr.predict(test_x)

### Valutazioni

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report

In [0]:
print("BASELINE MODEL", "\n")
print(confusion_matrix(test_y, yhat_dc))
print("\n")
print("Accuracy: ", accuracy_score(test_y, yhat_dc))
print("Precision: ", precision_score(test_y, yhat_dc))
print("Recall: ", recall_score(test_y, yhat_dc))
print("\n")
print(classification_report(test_y, yhat_dc))

In [0]:
print("LOGISTIC REGRESSION", "\n")
print(confusion_matrix(test_y, yhat_lr))
print("\n")
print("Accuracy: ", accuracy_score(test_y, yhat_lr))
print("Precision: ", precision_score(test_y, yhat_lr))
print("Recall: ", recall_score(test_y, yhat_lr))
print("\n")
print(classification_report(test_y, yhat_lr))

### Bilanciare il dataset

Effueremo un undersamplig completamente casuale, gli step sono:
* dividere il dataset secondo la variabile target
* creare un nuovo dataset con le osservazioni, estratte casualmente dal dataset più grande in modo che abbia lo stesso numero di ossevazioni della classe meno rappresentata
* concatenare

**N.B. Per l'esercizio abbiamo bilanciato perfettamente, nel mondo reale è più probabile che capiti di usare una percentuale diversa**

In [0]:
not_faud = df_complete[df_complete["isFraud"]==0].copy()
fraud = df_complete[df_complete["isFraud"]==1].copy()

In [0]:
not_fraud_under = not_faud.sample(fraud.shape[0], replace=True, random_state=42)

balanced_dataset = pd.concat([not_fraud_under, fraud], axis = 0)

balanced_dataset.isFraud.value_counts()

estraiamo train e test set, in questo caso non ci servirà stratificare

In [0]:
X_bal = balanced_dataset.drop("isFraud", axis=1).copy()
y_bal = balanced_dataset["isFraud"]

balanced_train_x, balanced_test_x, balanced_train_y, balanced_test_y = train_test_split(X_bal, y_bal, test_size = 0.3, random_state = 42)

### Modelling: dataset bilanciato

* baseline model
* reglog

In [0]:
# cambia la strategia del classificatore baseline, non più il più frequente visto che le classi sono bilanciate

dc_b = DummyClassifier()

dc_b.fit(balanced_train_x, balanced_train_y)

yhat_dc_b = dc_b.predict(balanced_test_x)

In [0]:
lr_b = LogisticRegression()

lr_b.fit(balanced_train_x, balanced_train_y)

yhat_lr_b = lr.predict(balanced_test_x)

### Valutazioni

In [0]:
print("BASELINE MODEL", "\n")
print(confusion_matrix(balanced_test_y, yhat_dc_b))
print("\n")
print("Accuracy: ", accuracy_score(balanced_test_y, yhat_dc_b))
print("Precision: ", precision_score(balanced_test_y, yhat_dc_b))
print("Recall: ", recall_score(balanced_test_y, yhat_dc_b))
print("\n")
print(classification_report(balanced_test_y, yhat_dc_b))

In [0]:
print("LOGISTIC REGRESSION", "\n")
print(confusion_matrix(balanced_test_y, yhat_lr_b))
print("\n")
print("Accuracy: ", accuracy_score(balanced_test_y, yhat_lr_b))
print("Precision: ", precision_score(balanced_test_y, yhat_lr_b))
print("Recall: ", recall_score(balanced_test_y, yhat_lr_b))
print("\n")
print(classification_report(balanced_test_y, yhat_lr_b))

**A lezione commenti sul risultato ed eventuali proposte di miglioramento**

### Citazioni

[PAYSIM: A FINANCIAL MOBILE MONEY SIMULATOR FOR FRAUD DETECTION](https://www.researchgate.net/publication/313138956_PAYSIM_A_FINANCIAL_MOBILE_MONEY_SIMULATOR_FOR_FRAUD_DETECTION)

### Link Utili

[Predicting Fraud in Financial Payment Services](https://www.kaggle.com/arjunjoshua/predicting-fraud-in-financial-payment-services)

[Logistic Regression — Detailed Overview](https://towardsdatascience.com/logistic-regression-detailed-overview-46c4da4303bc)

[Fix imbalanced dataset](https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb)